In [1]:
import onnx
import onnx_graphsurgeon as gs
import onnxruntime as ort
import torch
import numpy as np
from rich import print

In [2]:
def layer_wise(file,i1,i2):

    ort_session_1 = ort.InferenceSession(file)
    org_outputs = [x.name for x in ort_session_1.get_outputs()]

    model = onnx.load(file)
    for node in model.graph.node:
        for output in node.output:
            if output not in org_outputs:
                model.graph.output.extend([onnx.ValueInfoProto(name=output)])
    
    ort_session = ort.InferenceSession(model.SerializeToString())

    outputs = [x.name for x in ort_session.get_outputs()]
    inputs = [x.name for x in ort_session.get_inputs()]
   


    ort_outs = ort_session.run(None, input_feed={inputs[0]:i1,inputs[1]:i2})


    from collections import OrderedDict
    ort_outs = OrderedDict(zip(outputs, ort_outs))

    print(f"Input1 Shape :{i1.shape }Input1 Value :{i1}")
    print(f"Input1 Shape :{i2.shape }Input1 Value :{i2}")

    for key in ort_outs.keys():
        print(f"Layer : {key} Shape :{ort_outs[key].shape} Outputs :{ort_outs[key]}")
        

In [3]:
input_value1=np.random.randint(1,5,size=(2,2))
input_value2=np.random.randint(1,5,size=(2,2))

## Matrix Mul

In [4]:
i1 = gs.Variable("input1", np.int32, (2, 2))
i2 = gs.Variable("input2", np.int32, (2, 2))

matmul_node = gs.Node(op="MatMul",
                      name="MatMul_SumMul",
                      inputs=[i1, i2],
                      outputs=[gs.Variable("output_tensor", np.int32)])


graph = gs.Graph(nodes=[matmul_node], inputs=[i1,i2], outputs=[matmul_node.outputs[0]])

onnx_model = gs.export_onnx(graph)
onnx.save(onnx_model, "matrixmul.onnx")


In [5]:
layer_wise("matrixmul.onnx",input_value1,input_value2)

Input1 Shape :(2, 2)Input1 Value :[[1 2]
 [4 1]]

Input1 Shape :(2, 2)Input1 Value :[[1 2]
 [4 2]]

Layer : output_tensor Shape :(2, 2) Outputs :[[ 9  6]
 [ 8 10]]

## Approach 1 Matrix Mul using Mul and Reducesum 

In [6]:
i1 = gs.Variable("input1", np.int32, (2, 2))
i2 = gs.Variable("input2", np.int32, (2, 2))

unsqueeze_axes1 = gs.Constant(name="unsqueeze_axes1", values=np.array([2]))  
unsqueeze_axes2 = gs.Constant(name="unsqueeze_axes2", values=np.array([0]))  

unsqueeze_node1 = gs.Node (op="Unsqueeze",inputs=[i1],outputs=[gs.Variable(name="unsqueeze_output1", dtype=np.int32)], attrs={"axes": [2]})
unsqueeze_node2 = gs.Node(op="Unsqueeze",inputs=[i2],outputs=[gs.Variable(name="unsqueeze_output2", dtype=np.int32)], attrs={"axes": [0]})


mul_node = gs.Node(op="Mul", 
                   name="Mul_Node",
                   inputs=[unsqueeze_node1.outputs[0],unsqueeze_node2.outputs[0]], 
                   outputs=[gs.Variable(name="mul_output", dtype=np.int32)])


reduce_sum_node = gs.Node(op="ReduceSum", 
                          name="ReduceSum_Node",
                          inputs=[mul_node.outputs[0]], 
                          outputs=[gs.Variable(name="reduced_output", dtype=np.int32)],
                          attrs={"axes": [1]}) 

graph = gs.Graph(nodes=[unsqueeze_node1,unsqueeze_node2,mul_node,reduce_sum_node], 
                 inputs=[i1,i2], 
                 outputs=[reduce_sum_node.outputs[0]])


onnx_model = gs.export_onnx(graph)

onnx.save(onnx_model, "matmul_to_mul.onnx") 

In [7]:
layer_wise("matmul_to_mul.onnx",input_value1,input_value2)

Input1 Shape :(2, 2)Input1 Value :[[1 2]
 [4 1]]

Input1 Shape :(2, 2)Input1 Value :[[1 2]
 [4 2]]

Layer : reduced_output Shape :(2, 1, 2) Outputs :[[[ 9  6]]

 [[ 8 10]]]

Layer : unsqueeze_output1 Shape :(2, 2, 1) Outputs :[[[1]
  [2]]

 [[4]
  [1]]]

Layer : unsqueeze_output2 Shape :(1, 2, 2) Outputs :[[[1 2]
  [4 2]]]

Layer : mul_output Shape :(2, 2, 2) Outputs :[[[1 2]
  [8 4]]

 [[4 8]
  [4 2]]]

In [ ]:
import numpy as np

# Define the matrices
A = np.array([[1, 2],
              [3, 4]], dtype=np.float32)  # Shape: (2, 2)

B = np.array([[5, 6],
              [7, 8]], dtype=np.float32)  # Shape: (2, 2)


A_expanded = A[:, :, np.newaxis]   # Shape  (2, 2, 1)
B_expanded = B[np.newaxis, :, :]     # Shape  (1, 2, 2)

# print(f"{A_expanded}")
# print(f"{B_expanded}")



T = np.multiply(A_expanded,B_expanded)         # Shape: (2, 2, 2)
print(T)


C = np.sum(T, axis=1)                # Shape: (2, 2)

print("Result of MatMul via Element-wise Multiplication and Reduction:")
print(C)
print("Result of MatMul using MatMul")
print(np.matmul(A,B))


import numpy as np

def easy_product(m1, m2):
 
    m2_t = np.transpose(m2)  # Equivalent to your transpose function
    
    mats_p = np.array([[np.dot(row, col) for col in m2_t] for row in m1])

    return mats_p

print("Result of MatMul using Dot product")
print(easy_product(A,B))



[[[ 5.  6.]
  [14. 16.]]

 [[15. 18.]
  [28. 32.]]]

Result of MatMul via Element-wise Multiplication and Reduction:

[[19. 22.]
 [43. 50.]]

Result of MatMul using MatMul

[[19. 22.]
 [43. 50.]]

Result of MatMul using Dot product

[[19. 22.]
 [43. 50.]]

## Approach 2 Matrix Mul using Transpose Mul and Reducesum 

In [20]:
i1 = gs.Variable("input1", np.int32, (2, 2))
i2 = gs.Variable("input2", np.int32, (2, 2))


transpose_node=gs.Node(op="Transpose",name="Transpose1",inputs=[i2],outputs=[gs.Variable("trans_tensor", np.int32, (2, 2))])

mul_node = gs.Node(op="Mul", 
                   name="Mul_Node",
                   inputs=[i1,transpose_node.outputs[0]], 
                   outputs=[gs.Variable(name="mul_output", dtype=np.int32)])


reduce_sum_node = gs.Node(op="ReduceSum", 
                          name="ReduceSum_Node",
                          inputs=[mul_node.outputs[0]], 
                          outputs=[gs.Variable(name="reduced_output", dtype=np.int32)],
                          attrs={"axes": [0]}) 

graph = gs.Graph(nodes=[transpose_node,mul_node,reduce_sum_node], 
                 inputs=[i1,i2], 
                 outputs=[reduce_sum_node.outputs[0]])


onnx_model = gs.export_onnx(graph)

onnx.save(onnx_model, "matmul_to_mul_blog.onnx") 